In [33]:
import cv2

imgORG = cv2.imread('color_test1.bmp')

# imgGRAY = cv2.cvtColor(imgORG,cv2.COLOR_BGR2GRAY)
imgR = imgORG[:,:,2]
imgG = imgORG[:,:,1]
imgB = imgORG[:,:,0]

# cv2.namedWindow('image')
# cv2.imshow('image',imgR)
# cv2.waitKey()
# cv2.imshow('image',imgG)
# cv2.waitKey()
# cv2.imshow('image',imgB)
# cv2.waitKey()
# cv2.destroyAllWindows()

import numpy as np
# from matplotlib import pyplot as plt

imgBAY = np.zeros(imgR.shape)

imgBAY[0::2,0::2] = imgR[0::2,0::2]
imgBAY[0::2,1::2] = imgG[0::2,1::2]
imgBAY[1::2,0::2] = imgG[1::2,0::2]
imgBAY[1::2,1::2] = imgB[1::2,1::2]

imgBAY=imgBAY.astype('uint8')
print(imgBAY.shape)
cv2.imshow('image',imgBAY)
cv2.waitKey()
cv2.destroyAllWindows()

cv2.imwrite("imgBAY.bmp", imgBAY)

(748, 492)


True

In [ ]:
import cv2
import numpy as np

imgBAY = cv2.imread('imgBAY.bmp',0)
# imgRGB = cv2.cvtColor(imgBAY, cv2.COLOR_BAYER_BG2BGR)
imgRGB = np.zeros((imgBAY.shape[0], imgBAY.shape[1], 3), dtype=np.uint8)
#Pixel Doubling interpolation
#R
imgRGB[0::2,0::2,2] = imgBAY[0::2,0::2]
imgRGB[0::2,1::2,2] = imgBAY[0::2,0::2]
imgRGB[1::2,0::2,2] = imgBAY[0::2,0::2]
imgRGB[1::2,1::2,2] = imgBAY[0::2,0::2]
#G
imgRGB[0::2,0::2,1] = imgBAY[0::2,1::2]
imgRGB[0::2,1::2,1] = imgBAY[0::2,1::2]
imgRGB[1::2,0::2,1] = imgBAY[1::2,0::2]
imgRGB[1::2,1::2,1] = imgBAY[1::2,0::2]
#B
imgRGB[0::2,0::2,0] = imgBAY[1::2,1::2]
imgRGB[0::2,1::2,0] = imgBAY[1::2,1::2]
imgRGB[1::2,0::2,0] = imgBAY[1::2,1::2]
imgRGB[1::2,1::2,0] = imgBAY[1::2,1::2]


print(imgBAY.shape)
print(imgRGB.shape)
cv2.imshow('image',imgRGB)
cv2.waitKey()
cv2.destroyAllWindows()
cv2.imwrite("imgRGB_pixelDoubling.bmp", imgRGB)

(748, 492)
(748, 492, 3)


True

In [34]:
import cv2
import numpy as np

#마지막 인덱스
h = 747
w = 491

imgBAY = cv2.imread('imgBAY.bmp',0).astype(np.uint16)
# imgRGB = cv2.cvtColor(imgBAY, cv2.COLOR_BAYER_BG2BGR)
imgRGB = np.zeros((imgBAY.shape[0], imgBAY.shape[1], 3), dtype=np.uint16)
#Bilinear interpolation


def bilinear_int_R(rgb, arr):
	r, c = arr.shape
	for i in range(r):
		for j in range(c):
			i0 = abs(i-1); i1 = min(i+1, h)
			j0 = abs(j-1); j1 = min(j+1, w)
			if((i%2==0)and(j%2==0)):
				rgb[i,j,2] = arr[i,j]
			elif(i%2==0):
				rgb[i,j,2] = (arr[i,j0]+arr[i,j1])/2
			elif(j%2==0):
				rgb[i,j,2] = (arr[i0,j]+arr[i1,j])/2
			else:			
				rgb[i, j, 2] = (
					arr[i0, j0] + arr[i0, j1] +
					arr[i1, j0] + arr[i1, j1]
				) // 4.0

#G
def bilinear_int_G(rgb, arr):
	r, c = arr.shape
	for i in range(r):
		for j in range(c):
			i0 = abs(i-1); i1 = min(i+1, h)
			j0 = abs(j-1); j1 = min(j+1, w)
			if(((i%2==0)and(j%2==1))or((i%2==1)and(j%2==0))):
				rgb[i,j,1] = arr[i,j]
			else:
				rgb[i, j, 1] = (
					arr[i, j0] + arr[i, j1] +
					arr[i0, j] + arr[i1, j]
				) // 4.0


def bilinear_int_B(rgb, arr):
	r, c = arr.shape
	for i in range(r):
		for j in range(c):
			i0 = abs(i-1); i1 = min(i+1, h)
			j0 = abs(j-1); j1 = min(j+1, w)
			if((i%2==1)and(j%2==1)):
				rgb[i,j,0] = arr[i,j]
			elif(i%2==1):
				rgb[i,j,0] = (arr[i,j0]+arr[i,j1])//2
			elif(j%2==1):
				rgb[i,j,0] = (arr[i0,j]+arr[i1,j])//2
			else:			
				rgb[i, j, 0] = (
					arr[i0, j0] + arr[i0, j1] +
					arr[i1, j0] + arr[i1, j1]
				) // 4.0

bilinear_int_R(imgRGB, imgBAY)
bilinear_int_G(imgRGB, imgBAY)
bilinear_int_B(imgRGB, imgBAY)

out = np.clip(imgRGB, 0, 255).astype(np.uint8)

print(imgBAY.shape)
print(out.shape)
cv2.imshow('image',out)
cv2.waitKey()
cv2.destroyAllWindows()
cv2.imwrite("imgRGB_bilinear.bmp", out)

(748, 492)
(748, 492, 3)


True

In [ ]:
import cv2
import numpy as np

#마지막 인덱스
h = 747
w = 491

imgBAY = cv2.imread('imgBAY.bmp', 0).astype(np.float32)  # float 계산
H, W = imgBAY.shape
imgRGB = np.zeros((H, W, 3), dtype=np.float32)

#bicubic
def cubic(y_m1, y0, y1, y2):
    # Catmull-Rom @ 0.5 + anti-ringing clamp
    val = (-y_m1 + 9*y0 + 9*y1 - y2) / 16.0
    lo = min(y_m1, y0, y1, y2)
    hi = max(y_m1, y0, y1, y2)
    return np.clip(val, lo, hi)

def cubicR(arr,i,j):
	i0 = abs(i-3); i1 = abs(i-1)
	i2 = min(i+1,h); i3 = min(i+3, h)
	j0 = abs(j-3); j1 = abs(j-1)
	j2 = min(j+1,w); j3 = min(j+3, w)
	return cubic(arr[i,j0],arr[i,j1],
              		arr[i,j2],arr[i,j3])
def cubicC(arr,i,j):
	i0 = abs(i-3); i1 = abs(i-1)
	i2 = min(i+1,h); i3 = min(i+3, h)
	j0 = abs(j-3); j1 = abs(j-1)
	j2 = min(j+1,w); j3 = min(j+3, w)
	return cubic(arr[i0,j],arr[i1,j],
              		arr[i2,j],arr[i3,j])
 
def cubicCross(arr,i,j):
	i0 = abs(i-3); i1 = abs(i-1)
	i2 = min(i+1,h); i3 = min(i+3, h)
	j0 = abs(j-3); j1 = abs(j-1)
	j2 = min(j+1,w); j3 = min(j+3, w)
	# return cubic(cubicR(arr,i0,j),cubicR(arr,i1,j),
    #           cubicR(arr,i2,j),cubicR(arr,i3,j))
	return 0.5*(cubic(arr[i0,j0],arr[i1,j1],
              		arr[i2,j2],arr[i3,j3]) + 
             cubic(arr[i0,j3],arr[i1,j2],
              		arr[i2,j1],arr[i3,j0]))


def cubic_int_R(rgb, arr):
	r, c = arr.shape
	for i in range(r):
		for j in range(c):
			i0 = abs(i-3); i1 = abs(i-1)
			i2 = min(i+1,h); i3 = min(i+3, h)
			j0 = abs(j-3); j1 = abs(j-1)
			j2 = min(j+1,w); j3 = min(j+3, w)
   
			if((i%2==0)and(j%2==0)):
				rgb[i,j,2] = arr[i,j]
			elif(i%2==0):
				rgb[i,j,2] = cubicR(arr,i,j)
			elif(j%2==0):
				rgb[i,j,2] = cubicC(arr,i,j)
			else:			
				rgb[i,j,2] = cubicCross(arr,i,j)


def cubic_int_G(rgb, arr):
	r, c = arr.shape
	for i in range(r):
		for j in range(c):
			i0 = abs(i-3); i1 = abs(i-1)
			i2 = min(i+1,h); i3 = min(i+3, h)
			j0 = abs(j-3); j1 = abs(j-1)
			j2 = min(j+1,w); j3 = min(j+3, w)

   
			if(((i%2==1)and(j%2==0))or((i%2==0)and(j%2==1))):
				rgb[i,j,1] = arr[i,j]
			else:
				gh = abs(arr[i, j1] - arr[i, j2])
				gv = abs(arr[i1, j] - arr[i2, j])
				G_h = cubicR(arr, i, j)
				G_v = cubicC(arr, i, j)
				wh = 1.0/(gh + 1e-6); wv = 1.0/(gv + 1e-6)
				G = (wh*G_h + wv*G_v)/(wh + wv)
				imgRGB[i, j, 1] = G

def cubic_int_B(rgb, arr):
	r, c = arr.shape
	for i in range(r):
		for j in range(c):
			i0 = abs(i-3); i1 = abs(i-1)
			i2 = min(i+1,h); i3 = min(i+3, h)
			j0 = abs(j-3); j1 = abs(j-1)
			j2 = min(j+1,w); j3 = min(j+3, w)

   
			if((i%2==1)and(j%2==1)):
				rgb[i,j,0] = arr[i,j]
			elif((i%2==1)and(j%2==0)):
				rgb[i,j,0] = cubicR(arr,i,j)
			elif((i%2==0)and(j%2==1)):
				rgb[i,j,0] = cubicC(arr,i,j)
			else:			
				rgb[i,j,0] = cubicCross(arr,i,j)   




cubic_int_R(imgRGB, imgBAY)
cubic_int_G(imgRGB, imgBAY)
cubic_int_B(imgRGB, imgBAY)


out = np.clip(imgRGB, 0, 255).astype(np.uint8)

print(imgBAY.shape)
print(out.shape)
cv2.imshow('image',out)
cv2.waitKey()
cv2.destroyAllWindows()
cv2.imwrite("imgRGB_cubic.bmp", out)

6.0 8.0 86.0 86.6875 0.16666664 86.29465
0.0 11.0 89.0 84.3125 1000000.0 89.0
4.0 13.0 86.9375 80.3125 0.24999994 85.37868
1.0 10.0 85.0625 81.0625 0.99999905 84.69885
3.0 6.0 87.625 81.9375 0.33333322 85.729164
1.0 9.0 88.5 83.5 0.99999905 88.00001
3.0 7.0 89.375 84.4375 0.33333322 87.89375
1.0 7.0 91.75 86.75 0.99999905 91.12501
1.0 4.0 91.5 89.125 0.99999905 91.024994
1.0 9.0 91.25 87.6875 0.99999905 90.89376
3.0 11.0 93.75 89.9375 0.33333322 92.93303
3.0 5.0 93.625 89.625 0.33333322 92.12499
1.0 7.0 92.3125 87.25 0.99999905 91.67969
0.0 6.0 92.875 88.0 1000000.0 92.875
3.0 11.0 94.75 90.6875 0.33333322 93.87946
4.0 7.0 94.1875 90.4375 0.24999994 92.82386
0.0 5.0 92.0 91.4375 1000000.0 92.0
6.0 3.0 95.0625 92.3125 0.16666664 93.229164
1.0 5.0 97.8125 92.375 0.99999905 96.90626
1.0 6.0 97.4375 94.3125 0.99999905 96.99107
0.0 7.0 98.0 95.8125 1000000.0 98.0
3.0 4.0 96.75 94.125 0.33333322 95.62501
4.0 4.0 92.375 92.0 0.24999994 92.1875
7.0 5.0 94.375 89.375 0.14285712 91.458336
2.0 3.